**Augementation UDFs**

In [1]:
import cv2
import numpy as np

def rotation(image):
    angle = np.random.uniform(-30, 30) #rotate between -30 and 30 degrees
    h, w = image.shape[:2]
    M = cv2.getRotationMatrix2D((w / 2, h / 2), angle, 1)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_LINEAR)
    return rotated

def shift(image):
    max_shift = 25  # shift up to 25 pixels
    rows, cols = image.shape
    M = np.float32([[1, 0, np.random.randint(-max_shift, max_shift)],
                    [0, 1, np.random.randint(-max_shift, max_shift)]])
    shifted = cv2.warpAffine(image, M, (cols, rows))
    return shifted

def zoom(image):
    zoom_factor = np.random.uniform(1.0, 1.5)
    height, width = image.shape[:2]
    new_height, new_width = int(height * zoom_factor), int(width * zoom_factor)
    
    zoomed_image = cv2.resize(image, (new_width, new_height))
    crop_height = (new_height - height) // 2
    crop_width = (new_width - width) // 2
    zoomed_image = zoomed_image[crop_height:crop_height+height, crop_width:crop_width+width]
    return zoomed_image

def brightness(image):
    brightness_factor = np.random.uniform(0.5, 1.5)
    br_image = cv2.convertScaleAbs(image, alpha=brightness_factor, beta=0)
    return br_image

def blur(image):
    ksize = 3
    blurred_image = cv2.GaussianBlur(image, (ksize, ksize), 0)
    return blurred_image

def augment_image(image):
    if np.random.rand() < 0.5:
        image = rotation(image)
    if np.random.rand() < 0.5:
        image = shift(image)
    if np.random.rand() < 0.5:
        image = zoom(image)
    if np.random.rand() < 0.5:
        image = brightness(image)
    if np.random.rand() < 0.5:
        image = blur(image)
    return image

**Loading images and labels**

In [2]:
import os

def load_data(data_dir):
    images, labels = [], []
    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        for image_file in os.listdir(label_dir):
            image_path = os.path.join(label_dir, image_file)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (256, 256))
            images.append(image)
            labels.append(label)
    return np.array(images), np.array(labels)
    
def augment_data(images, labels):
    augmented_images, augmented_labels = [], []
    for image, label in zip(images, labels):
        augmented_images.append(image)
        augmented_labels.append(label)
        for i in range(5):
            aug_image = augment_image(image)
            augmented_images.append(aug_image)
            augmented_labels.append(label)
    return np.array(augmented_images), np.array(augmented_labels)

images, labels = load_data('ds')

**Extracting hog features**

In [3]:
from skimage.feature import hog
from sklearn.model_selection import train_test_split

def extract_hog_features(images):
    hog_features = []
    for image in images:
        features = hog(image, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(4, 4), block_norm='L2-Hys')
        hog_features.append(features)
    return np.array(hog_features)

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2)
aug_images, aug_labels = augment_data(X_train, y_train)

X_train = np.concatenate((X_train, aug_images))
y_train = np.concatenate((y_train, aug_labels))

X_train = extract_hog_features(X_train)
X_test = extract_hog_features(X_test)

**DS split**

In [4]:
#X_train, X_test, y_train, y_test = train_test_split(hog_features, labels, test_size=0.2) # stratify


**Model teaching**

In [4]:
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Encode labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# Train SVM model
svm_model = SVC(kernel='poly', C=4., gamma=1.,) #probability=True,
svm_model.fit(X_train, y_train_encoded)


SVC(C=4.0, gamma=1.0, kernel='poly')

**Predict and evaluate**

In [5]:
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test_encoded, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test_encoded, y_pred, target_names=le.classes_))

Accuracy: 0.946969696969697
              precision    recall  f1-score   support

        Fist       0.91      0.94      0.93        87
        Palm       0.99      0.98      0.99       118
       Thumb       0.91      0.88      0.90        59

    accuracy                           0.95       264
   macro avg       0.94      0.94      0.94       264
weighted avg       0.95      0.95      0.95       264



**Magic**

In [9]:
import pygame
import time 

pygame.mixer.init()

playlist = ['song1.mp3', 'song2.mp3']
current_song = 0

def play_music():
    global current_song
    pygame.mixer.music.load(playlist[current_song])
    pygame.mixer.music.play()

def stop_music():
    pygame.mixer.music.stop()
    
def skip_music():
    global current_song
    current_song = (current_song + 1) % len(playlist)
    play_music()

def recognize_gesture(frame, model, le):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.resize(gray_frame, (264, 264))
    features = hog(gray_frame, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(4, 4), block_norm='L2-Hys').reshape(1, -1)
    prediction = model.predict(features)
    label = le.inverse_transform(prediction)
    return label[0]

cnt_fist = 0
cnt_palm = 0
cnt_thumb = 0

def clear_cnt():
    cnt_fist = 0
    cnt_palm = 0
    cnt_thumb = 0
    
cap = cv2.VideoCapture(0)

play_music()

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    label = recognize_gesture(frame, svm_model, le)
    if label == 'Palm':
        cnt_palm += 1
        if cnt_palm > 10 :
            stop_music()
            clear_cnt()
            
    elif label == 'Thumb':
        cnt_thumb += 1
        if cnt_thumb > 10 :
            skip_music()
            clear_cnt()
            
    elif label == 'Fist':
        cnt_fist += 1
        if cnt_fist > 10 :
            play_music()
            clear_cnt()
    
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Gesture Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
pygame.mixer.quit()


2024-05-28 17:13:45.238 Python[18014:4749919] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.
